# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [24]:
import matplotlib.pyplot as plt
import os
import random
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

path = "Dataset"
print("Path to dataset files:", path)

Path to dataset files: Dataset


In [25]:
# Đường dẫn tới thư mục Dataset
base_path_real = "Dataset/Real1"
base_path_altered = "Dataset/Altered"

# Tìm tất cả các file BMP trong Dataset/Real và Dataset/Altered
all_images_real = [os.path.join(root, file) for root, _, files in os.walk(base_path_real) for file in files if file.endswith(".BMP")]
all_images_altered = [os.path.join(root, file) for root, _, files in os.walk(base_path_altered) for file in files if file.endswith(".BMP")]
all_images = all_images_real + all_images_altered

# Hàm xử lý và hiển thị hình ảnh
def process_and_plot(image_path, image_name):
    # Load ảnh vân tay
    image = cv2.imread(image_path)

    # Resize ảnh
    resized_image = cv2.resize(image, (0, 0), fx=7, fy=7, interpolation=cv2.INTER_CUBIC)

    # Loại bỏ nhiễu
    denoised_image = cv2.fastNlMeansDenoising(resized_image, None, 7, 7, 21)

    # Chuyển ảnh sang grayscale
    gray_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)

    # Làm mờ ảnh
    blurred_image = cv2.GaussianBlur(enhanced_image, (3, 3), 0)

    # Threshold thích nghi
    binary_image = cv2.adaptiveThreshold(enhanced_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Phát hiện cạnh
    edges = cv2.Canny(binary_image, threshold1=60, threshold2=120)

    # Tính toán trường hướng
    gradient_x = cv2.Sobel(enhanced_image, cv2.CV_64F, 1, 0, ksize=5)
    gradient_y = cv2.Sobel(enhanced_image, cv2.CV_64F, 0, 1, ksize=5)
    orientation_enhanced = cv2.phase(gradient_x, gradient_y, angleInDegrees=True)

    gradient_x = cv2.Sobel(binary_image, cv2.CV_64F, 1, 0, ksize=5)
    gradient_y = cv2.Sobel(binary_image, cv2.CV_64F, 0, 1, ksize=5)
    orientation_binary = cv2.phase(gradient_x, gradient_y, angleInDegrees=True)

    # Trích xuất và hiển thị điểm minutiae
    minutiae_visualization = np.dstack((binary_image, binary_image, binary_image))
    minutiae_points = []

    rows, cols = binary_image.shape
    for y in range(1, rows - 1):
        for x in range(1, cols - 1):
            if binary_image[y, x] == 255:
                neighborhood = binary_image[y - 1:y + 2, x - 1:x + 2]
                ridge_count = np.sum(neighborhood == 255)

                if ridge_count == 2:  # Ridge ending
                    minutiae_points.append((x, y))
                    cv2.circle(minutiae_visualization, (x, y), 3, (0, 0, 255), -1)
                elif ridge_count == 4:  # Bifurcation
                    minutiae_points.append((x, y))
                    cv2.circle(minutiae_visualization, (x, y), 3, (0, 255, 0), -1)

    # Lưu kết quả vào thư mục tương ứng
    if base_path_altered in image_path:
        output_dir = "Output/Altered"
    else:
        output_dir = "Output/Real1"

    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_minutiae_visualization.jpg"), minutiae_visualization)
'''
    # Hiển thị kết quả
    fig, axes = plt.subplots(1, 9, figsize=(18, 5))

    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    axes[1].imshow(gray_image, cmap="gray")
    axes[1].set_title("Grayscale Image")
    axes[1].axis("off")

    axes[2].imshow(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB))
    axes[2].set_title("Blurred Image")
    axes[2].axis("off")

    axes[3].imshow(edges, cmap="gray")
    axes[3].set_title("Edges")
    axes[3].axis("off")

    axes[4].imshow(enhanced_image, cmap="gray")
    axes[4].set_title("Enhanced Image")
    axes[4].axis("off")

    axes[5].imshow(orientation_enhanced, cmap="gray")
    axes[5].set_title("Orientation Enhanced")
    axes[5].axis("off")

    axes[6].imshow(binary_image, cmap="gray")
    axes[6].set_title("Binary Image")
    axes[6].axis("off")

    axes[7].imshow(orientation_binary, cmap="gray")
    axes[7].set_title("Orientation Binary")
    axes[7].axis("off")

    axes[8].imshow(cv2.cvtColor(minutiae_visualization, cv2.COLOR_BGR2RGB))
    axes[8].set_title("Minutiae Points")
    axes[8].axis("off")

    fig.suptitle(image_name, fontsize=14, y=0.02)
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.show()
'''
# Xử lý tất cả ảnh
for img_path in all_images:
    img_name = os.path.basename(img_path)
    process_and_plot(img_path, img_name)

# Xử lý tất cả ảnh với thanh tiến độ
print("Processing all images...")
for img_path in tqdm(all_images, desc="Progress", unit="image"):
    img_name = os.path.basename(img_path)
    process_and_plot(img_path, img_name)

print("Processing completed!")


Processing all images...


Progress: 100%|██████████| 40/40 [01:21<00:00,  2.05s/image]

Processing completed!


In [26]:
output_path_real = "Output/Real1"
output_path_altered = "Output/Altered"

# Tìm tất cả các file minutiae visualization trong Output/Real và Output/Altered
all_images_real = [os.path.join(root, file) for root, _, files in os.walk(output_path_real) for file in files if file.endswith("_minutiae_visualization.jpg")]
all_images_altered = [os.path.join(root, file) for root, _, files in os.walk(output_path_altered) for file in files if file.endswith("_minutiae_visualization.jpg")]

# Kiểm tra dữ liệu
if not all_images_real:
    raise ValueError("No images found in Real folder.")
if not all_images_altered:
    raise ValueError("No images found in Altered folder.")

# Lưu trữ feature và label
features_train = []
labels_train = []
features_test = []
labels_test = []

# Hàm trích xuất minutiae feature
def extract_minutiae_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(image)

    binary_image = cv2.adaptiveThreshold(enhanced_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    minutiae_points = []

    rows, cols = binary_image.shape
    for y in range(1, rows - 1):
        for x in range(1, cols - 1):
            if binary_image[y, x] == 255:
                neighborhood = binary_image[y - 1:y + 2, x - 1:x + 2]
                ridge_count = np.sum(neighborhood == 255)

                if ridge_count == 2 or ridge_count == 4:  # Ridge ending or bifurcation
                    minutiae_points.append((x, y))

    return [len(minutiae_points), np.mean(binary_image), np.std(binary_image)]

# Gán nhãn cho từng ngón từ thư mục Real
finger_labels = {}
for idx, img_path in enumerate(all_images_real):
    finger_id = "_".join(os.path.basename(img_path).split("_")[2:4])  # Lấy ngón tay từ tên file
    if finger_id not in finger_labels:
        finger_labels[finger_id] = len(finger_labels)  # Gán nhãn số duy nhất cho mỗi ngón tay
    features_train.append(extract_minutiae_features(img_path))
    labels_train.append(finger_labels[finger_id])  # Gán nhãn số cho mỗi ngón tay

# Xử lý tất cả ảnh Altered (Test)
for img_path in all_images_altered:
    # Bỏ hậu tố (_CR, _Obl, _Zcut) để khớp với nhãn từ Real
    base_name = os.path.basename(img_path).split("_")
    finger_id = "_".join(base_name[2:4])  # Lấy ngón tay từ tên file
    if finger_id in finger_labels:
        features_test.append(extract_minutiae_features(img_path))
        labels_test.append(finger_labels[finger_id])  # Gán nhãn nếu tìm thấy trong Real

# Kiểm tra số lớp trong dữ liệu huấn luyện
if len(set(labels_train)) < 2:
    raise ValueError("Training data must have at least two classes. Found only one class.")

# Kiểm tra dữ liệu test
if not features_test:
    raise ValueError("No matching IDs found between Real and Altered datasets.")

# Chuyển đổi danh sách thành mảng numpy
features_train = np.array(features_train)
features_test = np.array(features_test)

# Kiểm tra nếu các đặc trưng rỗng
if features_train.size == 0 or features_test.size == 0:
    raise ValueError("Feature extraction failed. Ensure the input images are valid and correctly formatted.")

# Xây dựng model SVM
model = svm.SVC(kernel='linear', C=1.0)
model.fit(features_train, labels_train)

# Dự đoán và đánh giá
y_pred = model.predict(features_test)
print("Accuracy:", accuracy_score(labels_test, y_pred))
print("Classification Report:\n", classification_report(labels_test, y_pred))

print("Processing and modeling completed!")



Accuracy: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.13      0.21        15
           1       0.50      0.87      0.63        15

    accuracy                           0.50        30
   macro avg       0.50      0.50      0.42        30
weighted avg       0.50      0.50      0.42        30

Processing and modeling completed!


In [21]:
# Hàm tìm ảnh phù hợp nhất từ Real
def find_best_match_by_minutiae(altered_image_path):
    altered_features = extract_minutiae_features(altered_image_path)

    best_match_path = None
    best_match_count = 0

    for root, _, files in os.walk(base_path_real):
        for file in files:
            if file.endswith(".BMP"):
                real_image_path = os.path.join(root, file)
                real_features = extract_minutiae_features(real_image_path)

                # Đếm số điểm chung giữa Altered và Real
                match_count = len(altered_features & real_features)

                if match_count > best_match_count:
                    best_match_count = match_count
                    best_match_path = real_image_path

    return best_match_path, best_match_count
'''
# Hiển thị danh sách file trong Dataset/Altered
print("Available altered fingerprint images:")
altered_files = [os.path.basename(file) for file in os.listdir(base_path_altered) if file.endswith(".BMP")]
if not altered_files:
    raise FileNotFoundError("No altered fingerprint images found in the Dataset/Altered folder.")
print("\n".join(altered_files))
'''
# Nhập tên file ảnh từ Altered
input_altered_image_name = input("Enter the name of the altered fingerprint image: ")
altered_image_path = os.path.join(base_path_altered, input_altered_image_name)

if not os.path.exists(altered_image_path):
    raise FileNotFoundError(f"Altered image {input_altered_image_name} not found. Ensure it exists in Dataset/Altered.")

# Tìm ảnh phù hợp nhất
best_match_path, match_count = find_best_match_by_minutiae(altered_image_path)

if best_match_path:
    print(f"Best match found: {os.path.basename(best_match_path)}")
    print(f"Matching minutiae points: {match_count}")
else:
    print("No match found in Real dataset.")


Best match found: 1__M_Left_middle_finger.BMP
Matching minutiae points: 814
